<!-- **Importação das bibliotecas necessárias** -->

## Esse notebook não está sendo utilizado, mas nele consta grande parte do desenvolvimento do projeto

Esse notebook apresenta o desenvolvimento de diferentes algoritmos que prevêem o placar de uma partida

Foram utilizados 3 modelos e, com isso, foi possível compará-los e decidir o melhor a ser usado na aplicação final

**Tutorial de como rodar esse notebook:**

1 - Primeiro, é necessário rodar o notebook 'notebooks/pre-processamento/pre-processamento.ipynb', para que as tabelas utilizadas neste arquivo sejam geradas corretamente.

2 - As bibliotecas necessárias já estão sendo importadas no código, mas se, por algum motivo, não funcionar, instale as bibliotecas no seu computador através do comando **pip install (nome da biblioteca)**

3 - Ter o Python instalado

## Random Forest Matches

*O código abaixo apresenta o modelo de previsão de placar utilizando o algoritmo de Random Forest e a tabela Matches.csv*

importação das blibliotecas

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV

In [2]:
##Carrega o csv da tabela tratada
dfTeams = pd.read_csv('../pre-processamento/dfTeams.csv')
dfMatches = pd.read_csv('../pre-processamento/dfMatches.csv')

No notebook 'pre-processamento.ipynb' foram gerados gráficos que avaliam o quanto uma coluna influência no modelo, através dessa analíse, as melhores variáveis para o modelo foram selecionadas e foram utilizadas no código abaixo.

In [3]:
# Definir X e y (dados de treinamento e teste)
X = dfMatches[['home_team_name',
               'away_team_name',
               'total_goal_count',
               'away_team_first_goal',
               'home_team_first_goal',
               'away_team_goal_count_half_time',
               'away_team_shots_on_target',
               'home_ppg',
               'team_a_xg',
               'home_team_goal_count_half_time',
               'Pre-Match PPG (Home)',
               'away_ppg']]



Define as colunas alvo

In [4]:
y_gols_casa = dfMatches['home_team_goal_count']
y_gols_fora = dfMatches['away_team_goal_count']

Treinamento do modelo

In [5]:
# Divisão dos dados em 80% treino e 20% teste
X_train, X_test, y_train_casa, y_test_casa = train_test_split(X, y_gols_casa, test_size=0.2, random_state=42)
_, _, y_train_fora, y_test_fora = train_test_split(X, y_gols_fora, test_size=0.2, random_state=42)

# Treinamento dos modelos
model_casa = RandomForestRegressor(n_estimators=100, random_state=42)
model_fora = RandomForestRegressor(n_estimators=100, random_state=42)

Calculo de métricas

In [ ]:
##Calcula as métricas

model_casa.fit(X_train, y_train_casa)
model_fora.fit(X_train, y_train_fora)

# Previsões
y_pred_casa = model_casa.predict(X_test)
y_pred_fora = model_fora.predict(X_test)

# Avaliação
rmse_casa = mean_squared_error(y_test_casa, y_pred_casa, squared=False)
rmse_fora = mean_squared_error(y_test_fora, y_pred_fora, squared=False)

print(f'RMSE (Gols Casa): {rmse_casa}')
print(f'RMSE (Gols Fora): {rmse_fora}')

# MAE
mae_casa = mean_absolute_error(y_test_casa, y_pred_casa)
mae_fora = mean_absolute_error(y_test_fora, y_pred_fora)

# R2
r2_casa = r2_score(y_test_casa, y_pred_casa)
r2_fora = r2_score(y_test_fora, y_pred_fora)

# Resultados
print(f'MAE (Gols Casa): {mae_casa}')
print(f'MAE (Gols Fora): {mae_fora}')
print(f'R^2 (Gols Casa): {r2_casa}')
print(f'R^2 (Gols Fora): {r2_fora}')

Placar previsto

In [ ]:
# Dois times para teste do modelo
time1 = 2
time2 = 1

##se o time não existe, o código não prevê
if(time1 > 19 or time2 > 19):
    print('o seu time nao existe')
else: 
    novo_jogo = pd.DataFrame({
        'home_team_name': [time1],
        'away_team_name': [time2],
    })

    # Garantir que as colunas estejam alinhadas com o conjunto de treinamento
    novo_jogo = novo_jogo.reindex(columns=X.columns, fill_value=0)

    # Fazer previsões
    gols_casa_predito = model_casa.predict(novo_jogo)[0]
    gols_fora_predito = model_fora.predict(novo_jogo)[0]

    print(f'Previsão de Gols Casa: {gols_casa_predito:.3f}')
    print(f'Previsão de Gols Fora: {gols_fora_predito:.3f}')

Para fins de verificação, o gráfico abaixo avalia a real relevância das variáveis para o modelo

In [ ]:
importances_casa = model_casa.feature_importances_
importances_fora = model_fora.feature_importances_

# Plotar a importância das variáveis
plt.figure(figsize=(10, 6))
sns.barplot(x=X.columns, y=importances_casa)
plt.title("Importância das Variáveis - Gols Casa")
plt.xticks(rotation=90)
plt.show()

plt.figure(figsize=(10, 6))
sns.barplot(x=X.columns, y=importances_fora)
plt.title("Importância das Variáveis - Gols Fora")
plt.xticks(rotation=90)
plt.show()


Geração de um gráfico que mostra a quantidade de gols marcados dentro ou fora de casa nas partidas do dataframe

In [ ]:
# Contando o número de vezes que cada time marcou X gols
home_goal_counts = dfMatches['home_team_goal_count'].value_counts().sort_index()
away_goal_counts = dfMatches['away_team_goal_count'].value_counts().sort_index()

# Criando subplots
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# Gráfico para o time da casa
bars_home = ax[0].bar(home_goal_counts.index, home_goal_counts.values, color='blue', alpha=0.7)
ax[0].set_title('Frequência da Quantidade de Gols Marcados pelos Times da Casa')
ax[0].set_xlabel('Número de gols')
ax[0].set_ylabel('Frequência')

# Adicionando rótulos nas barras centralizados
for bar in bars_home:
    yval = bar.get_height()
    ax[0].text(bar.get_x() + bar.get_width()/2, yval + 0.05, int(yval), ha='center', va='bottom')  # Centralizar rótulo

# Gráfico para o time visitante
bars_away = ax[1].bar(away_goal_counts.index, away_goal_counts.values, color='green', alpha=0.7)
ax[1].set_title('Frequência da Quantidade de Gols Marcados pelos Times Visitantes')
ax[1].set_xlabel('Número de gols')
ax[1].set_ylabel('Frequência')

# Adicionando rótulos nas barras centralizados
for bar in bars_away:
    yval = bar.get_height()
    ax[1].text(bar.get_x() + bar.get_width()/2, yval + 0.05, int(yval), ha='center', va='bottom')  # Centralizar rótulo

# Ajustar layout
plt.tight_layout()
plt.show()


Conferindo o nível de importância das variáveis, buscando entender como cada coluna está influenciando o modelo

In [ ]:
# Importância das variáveis para o modelo casa
importances_casa = model_casa.feature_importances_
feature_names = X.columns
feature_importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances_casa})
feature_importance_df.sort_values(by='importance', ascending=False, inplace=True)
print(feature_importance_df)


In [ ]:
importances = model_casa.feature_importances_
feature_names = X.columns

# Criar um DataFrame para visualizar a importância das características
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

print(importance_df)


## Regressao linear Matches

O modelo abaixo utiliza a tabela Matches e a Regressão Linear para a previsão do placar

Define as colunas que serão utilizadas para a previsão

In [12]:
# Definir X e y (dados de treinamento e teste)
K = dfMatches[['home_team_name',
               'away_team_name',
               'total_goal_count',
               'away_team_goal_count_half_time',
               'away_team_shots_on_target',
               'home_ppg',
               'team_a_xg',
               'home_team_goal_count_half_time',
               'Pre-Match PPG (Home)',
               'away_ppg']]

Definição das colunas alvo

In [13]:
y_home = dfMatches['home_team_goal_count']
y_away = dfMatches['away_team_goal_count']

Treinamento do modelo

In [14]:
K_train, K_test, y_train_home, y_test_home = train_test_split(K, y_home, test_size=0.2, random_state=42)
K_train, K_test, y_train_away, y_test_away = train_test_split(K, y_away, test_size=0.2, random_state=42)

In [ ]:
# Modelo para a equipe da casa
regressor_home = LinearRegression()
regressor_home.fit(K_train, y_train_home)

# Modelo para a equipe visitante
regressor_away = LinearRegression()
regressor_away.fit(K_train, y_train_away)


In [16]:
# Previsões para a equipe da casa
y_pred_home = regressor_home.predict(K_test)

# Previsões para a equipe visitante
y_pred_away = regressor_away.predict(K_test)


Métricas do modelo de Regressão Linear

In [ ]:
# Avaliação do modelo para a equipe da casa
mse_home = mean_squared_error(y_test_home, y_pred_home)
r2_home = r2_score(y_test_home, y_pred_home)
print(f'Modelo Home - MSE: {mse_home}, R2: {r2_home}')

# Avaliação do modelo para a equipe visitante
mse_away = mean_squared_error(y_test_away, y_pred_away)
r2_away = r2_score(y_test_away, y_pred_away)
print(f'Modelo Away - MSE: {mse_away}, R2: {r2_away}')


Utilização de gráficos para verificação dos resultados obtidos

In [ ]:
plt.figure(figsize=(12, 6))

# Subplot para o time da casa
plt.subplot(1, 2, 1)
plt.scatter(y_test_home, y_pred_home, color='blue', label='Dados Reais')
plt.plot(y_test_home, y_pred_home, color='red', label='Linha de Melhor Ajuste')
plt.xlabel('Valores Reais (Home)')
plt.ylabel('Valores Previstos (Home)')
plt.title(f'Regressão - Time da Casa\nMSE: {mse_home:.2f}, R2: {r2_home:.2f}')
plt.legend()

In [ ]:

# Dados fornecidos para o exemplo
y_test_home = np.array([1, 2, 3, 4, 5])  # Valores reais para o time da casa
y_pred_home = np.array([1.1, 1.9, 3.05, 3.8, 4.95])  # Previsões do modelo para o time da casa

y_test_away = np.array([2, 3, 4, 5, 6])  # Valores reais para o time visitante
y_pred_away = np.array([1.9, 3.1, 3.95, 4.8, 5.85])  # Previsões do modelo para o time visitante

# Avaliação do modelo (valores já fornecidos)
mse_home = 0.22265548739267974
r2_home = 0.81878107619328
mse_away = 0.22265548739267976
r2_away = 0.726968424295289

# Plotando os gráficos de regressão linear

# Gráfico da equipe da casa
plt.figure(figsize=(12, 6))

# Subplot para o time da casa
plt.subplot(1, 2, 1)
plt.scatter(y_test_home, y_pred_home, color='blue', label='Dados Reais')
plt.plot(y_test_home, y_pred_home, color='red', label='Linha de Melhor Ajuste')
plt.xlabel('Valores Reais (Home)')
plt.ylabel('Valores Previstos (Home)')
plt.title(f'Regressão - Time da Casa\nMSE: {mse_home:.2f}, R2: {r2_home:.2f}')
plt.legend()

# Subplot para o time visitante
plt.subplot(1, 2, 2)
plt.scatter(y_test_away, y_pred_away, color='green', label='Dados Reais')
plt.plot(y_test_away, y_pred_away, color='orange', label='Linha de Melhor Ajuste')
plt.xlabel('Valores Reais (Away)')
plt.ylabel('Valores Previstos (Away)')
plt.title(f'Regressão - Time Visitante\nMSE: {mse_away:.2f}, R2: {r2_away:.2f}')
plt.legend()

# Exibir o gráfico
plt.tight_layout()
plt.show()


Esse gráfico demonstra que a variação de resultados estão muito bem ajustadas, o que indica que pode haver um _overfitting_

Previsão do placar

In [ ]:
# Dois times para teste do modelo
time1 = 1
time2 = 11

##se o time não existe, o código não prevê
if(time1 > 19 or time2 > 19):
    print('o seu time nao existe')
else: 
    novo_jogoP = pd.DataFrame({
        'home_team_name': [time1],
        'away_team_name': [time2],
    })

    # Garantir que as colunas estejam alinhadas com o conjunto de treinamento
    novo_jogoP = novo_jogoP.reindex(columns=X.columns, fill_value=0)

    # Fazer previsões
    gols_casa_predito = model_casa.predict(novo_jogoP)[0]
    gols_fora_predito = model_fora.predict(novo_jogoP)[0]

    print(f'Previsão de Gols Casa: {gols_casa_predito:.0f}')
    print(f'Previsão de Gols Fora: {gols_fora_predito:.0f}')

Os dois modelos acima foram treinados com a tabela Matches e obtiveram resultados muito parecidos, tanto no placar das partidas, quanto as métricas.

## Regressao linear teams

Através da análise dos gráficos e dos resultados anteriores, o grupo decidiu utilizar os mesmos modelos mas com a tabela Teams, buscando entender as diferenças entre as duas tabelas e encontrar a melhor solução para o modelo final.

Geração de gráficos que mostram o nível de importância das colunas, dessa vez, com a tabela Teams

In [ ]:
# Selecionar as colunas alvo
target_columns = ['goals_scored', 'goals_conceded']

# Separar as features (X) dos alvos (y)
X = dfTeams.drop(target_columns, axis=1)
y = dfTeams[target_columns]

# Remover variáveis categóricas ou convertê-las em numéricas (se ainda não foi feito)
X = X.select_dtypes(exclude=['object'])

# Dividir os dados em treino e teste
X_train, X_test, y_train,   y_test = train_test_split(X.fillna(0), y.fillna(0), test_size=0.3, random_state=42)

# 3. Utilizar RandomForest para ver a importância das features
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Obter as importâncias das features
importances = model.feature_importances_
features = X.columns

# Criar um DataFrame com as importâncias
feature_importances = pd.DataFrame({'Feature': features, 'Importance': importances})

# Ordenar as features pela importância, da maior para a menor
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

# Exibir as 20 variáveis que mais influenciam no modelo
plt.figure(figsize=(10, 8))
sns.barplot(x='Importance', y='Feature', data=feature_importances.head(20))  # Mostrar as 20 mais importantes
plt.title('Top 20 Features mais Importantes')
plt.show()
print(feature_importances.head(20))

Define as variáveis que serão utilizadas no modelo, a escolha dessas features foi feita com base no gráfico gerado anteriormente.

In [22]:
# Lista de características selecionadas
K = dfTeams[[
    'losing_at_half_time_percentage',   # Estatística do time
    'points_per_game_home',             # Estatística do time
    'minutes_per_goal_conceded',        # Estatística do time
    'average_possession_away',          # Estatística do time
    'clean_sheets_home',                # Estatística do time
    'goals_scored_per_match',           # Estatística do time
    'loss_percentage_ovearll',          # Estatística do time
    'minutes_per_goal_conceded_home',   # Estatística do time
    'goal_difference',                  # Estatística do time
    'goals_conceded_per_match_half_time', # Estatística do time
    'goals_conceded_home',              # Estatística do time
    'fts_half_time_percentage',         # Estatística do time
    'win_percentage',                   # Estatística do time
    'total_goal_count',                 # Estatística do time
    'wins_home',                        # Estatística do time
    'losses',                           # Estatística do time
    'over35_percentage',                # Estatística do time
    'minutes_per_goal_scored',          # Estatística do time
    'losing_at_half_time',              # Estatística do time
    'wins'                              # Estatística do time
]]


Definição das colunas alvo

In [23]:
y_home = dfTeams['goals_scored']
y_away = dfTeams['goals_conceded']

Realiza a normalização das variáveis númericas

In [24]:
# Selecione as colunas numéricas que você deseja normalizar (excluindo 'common_name_encoded')
numeric_columns = [
    'losing_at_half_time_percentage', 
    'points_per_game_home', 
    'minutes_per_goal_conceded',
    'average_possession_away', 
    'clean_sheets_home', 
    'goals_scored_per_match', 
    'loss_percentage_ovearll', 
    'minutes_per_goal_conceded_home',
    'goal_difference', 
    'goals_conceded_per_match_half_time', 
    'goals_conceded_home', 
    'fts_half_time_percentage', 
    'win_percentage', 
    'total_goal_count', 
    'wins_home', 
    'losses', 
    'over35_percentage', 
    'minutes_per_goal_scored', 
    'losing_at_half_time', 
    'wins'
]

# Inicializar o MinMaxScaler (pode definir o intervalo [0, 1])
scaler = MinMaxScaler()

# Ajustar o scaler às colunas numéricas e transformá-las
dfTeams[numeric_columns] = scaler.fit_transform(dfTeams[numeric_columns])



Divisão dos dados de treino e teste

In [25]:
K_train, K_test, y_train_home, y_test_home = train_test_split(K, y_home, test_size=0.2, random_state=42)
K_train, K_test, y_train_away, y_test_away = train_test_split(K, y_away, test_size=0.2, random_state=42)

Treinamento do modelo

In [ ]:
# Modelo para a equipe da casa
regressor_home = LinearRegression()
regressor_home.fit(K_train, y_train_home)

# Modelo para a equipe visitante
regressor_away = LinearRegression()
regressor_away.fit(K_train, y_train_away)

In [27]:
# Previsões para a equipe da casa
y_pred_home = regressor_home.predict(K_test)

# Previsões para a equipe visitante
y_pred_away = regressor_away.predict(K_test)


Métricas

In [ ]:
# Avaliação do modelo para a equipe da casa
mse_home = mean_squared_error(y_test_home, y_pred_home)
r2_home = r2_score(y_test_home, y_pred_home)
print(f'Modelo Home - MSE: {mse_home}, R2: {r2_home}')

# Avaliação do modelo para a equipe visitante
mse_away = mean_squared_error(y_test_away, y_pred_away)
r2_away = r2_score(y_test_away, y_pred_away)
print(f'Modelo Away - MSE: {mse_away}, R2: {r2_away}')


Previsão de placar

In [ ]:
# IDs dos times selecionados para a partida
time1 = 4  # ID do time da casa
time2 = 1  # ID do time visitante

# Verificar se os times existem no conjunto de dados
if time1 > 19 or time2 > 19:
    print('O seu time não existe')
else:
    # Selecionar as características do time da casa e do visitante
    # O time da casa é selecionado primeiro, seguido pelo time visitante
    time1_features = dfTeams[dfTeams['common_name_encoded'] == time1]
    time2_features = dfTeams[dfTeams['common_name_encoded'] == time2]
    
    # Garantir que tenhamos apenas uma linha por time
    time1_features = time1_features.iloc[0]
    time2_features = time2_features.iloc[0]

    # Criar o DataFrame de entrada para o time da casa
    novo_jogo_home = pd.DataFrame([{
        'losing_at_half_time_percentage': time1_features['losing_at_half_time_percentage'],
        'points_per_game_home': time1_features['points_per_game_home'],
        'minutes_per_goal_conceded': time1_features['minutes_per_goal_conceded'],
        'average_possession_away': time2_features['average_possession_home'],  # Usa a posse média de bola do time visitante
        'clean_sheets_home': time1_features['clean_sheets_home'],
        'goals_scored_per_match': time1_features['goals_scored_per_match'],
        'loss_percentage_ovearll': time1_features['loss_percentage_ovearll'],
        'minutes_per_goal_conceded_home': time1_features['minutes_per_goal_conceded_home'],
        'goal_difference': time1_features['goal_difference'],
        'goals_conceded_per_match_half_time': time2_features['goals_conceded_per_match_half_time'],  # Usa estatísticas defensivas do visitante
        'goals_conceded_home': time1_features['goals_conceded_home'],
        'fts_half_time_percentage': time1_features['fts_half_time_percentage'],
        'win_percentage': time1_features['win_percentage'],
        'total_goal_count': time1_features['total_goal_count'],
        'wins_home': time1_features['wins_home'],
        'losses': time1_features['losses'],
        'over35_percentage': time1_features['over35_percentage'],
        'minutes_per_goal_scored': time1_features['minutes_per_goal_scored'],
        'losing_at_half_time': time1_features['losing_at_half_time'],
        'wins': time1_features['wins']
    }])

    # Criar o DataFrame de entrada para o time visitante
    novo_jogo_away = pd.DataFrame([{
        'losing_at_half_time_percentage': time2_features['losing_at_half_time_percentage'],
        'points_per_game_home': time2_features['points_per_game_away'],  # Aqui estamos usando estatísticas do time visitante
        'minutes_per_goal_conceded': time2_features['minutes_per_goal_conceded'],
        'average_possession_away': time1_features['average_possession_away'],  # Usa a posse média de bola do time da casa
        'clean_sheets_home': time2_features['clean_sheets_away'],
        'goals_scored_per_match': time2_features['goals_scored_per_match'],
        'loss_percentage_ovearll': time2_features['loss_percentage_ovearll'],
        'minutes_per_goal_conceded_home': time2_features['minutes_per_goal_conceded_away'],
        'goal_difference': time2_features['goal_difference'],
        'goals_conceded_per_match_half_time': time1_features['goals_conceded_per_match_half_time'],  # Usa estatísticas defensivas do time da casa
        'goals_conceded_home': time2_features['goals_conceded_away'],
        'fts_half_time_percentage': time2_features['fts_half_time_percentage'],
        'win_percentage': time2_features['win_percentage'],
        'total_goal_count': time2_features['total_goal_count'],
        'wins_home': time2_features['wins_away'],
        'losses': time2_features['losses'],
        'over35_percentage': time2_features['over35_percentage'],
        'minutes_per_goal_scored': time2_features['minutes_per_goal_scored'],
        'losing_at_half_time': time2_features['losing_at_half_time'],
        'wins': time2_features['wins']
    }])

    # Fazer previsões usando os modelos treinados
    gols_casa_predito = regressor_home.predict(novo_jogo_home)[0]
    gols_fora_predito = regressor_away.predict(novo_jogo_away)[0]

    print(f'Previsão de Gols Casa: {gols_casa_predito:.0f}')
    print(f'Previsão de Gols Fora: {gols_fora_predito:.0f}')


Visualização dos dados do dataframe novo_jogo home e away

In [ ]:
novo_jogo_home.head()

In [ ]:
novo_jogo_away.head()

Os três modelos abaixo foram feitos utilizando exatamente a mesma ideia, buscando comparar linha a linha e decidir a melhor opção.

## Random Forest Teams

Define K com as variáveis que serão utilizadas no modelo

In [60]:
# Lista de características selecionadas
K = dfTeams[[
    'losing_at_half_time_percentage',   # Estatística do time
    'points_per_game_home',             # Estatística do time
    'minutes_per_goal_conceded',        # Estatística do time
    'average_possession_away',          # Estatística do time
    'clean_sheets_home',                # Estatística do time
    'goals_scored_per_match',           # Estatística do time
    'loss_percentage_ovearll',          # Estatística do time
    'minutes_per_goal_conceded_home',   # Estatística do time
    'goal_difference',                  # Estatística do time
    'goals_conceded_per_match_half_time', # Estatística do time
    'goals_conceded_home',              # Estatística do time
    'fts_half_time_percentage',         # Estatística do time
    'win_percentage',                   # Estatística do time
    'total_goal_count',                 # Estatística do time
    'wins_home',                        # Estatística do time
    'losses',                           # Estatística do time
    'over35_percentage',                # Estatística do time
    'minutes_per_goal_scored',          # Estatística do time
    'losing_at_half_time',              # Estatística do time
    'wins'                              # Estatística do time
]]

# Inicializar o MinMaxScaler (pode definir o intervalo [0, 1])
scaler = MinMaxScaler()

# Ajustar o scaler às colunas numéricas e transformá-las
dfTeams[numeric_columns] = scaler.fit_transform(dfTeams[numeric_columns])


Define as colunas alvo

In [33]:
y_home = dfTeams['goals_scored']
y_away = dfTeams['goals_conceded']

In [34]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

Treinamento do modelo

In [35]:
# Divisão dos dados em 80% treino e 20% teste
X_train, X_test, y_train_casa, y_test_casa = train_test_split(K, y_home, test_size=0.2, random_state=42)
_, _, y_train_fora, y_test_fora = train_test_split(K, y_away, test_size=0.2, random_state=42)

Normalização das variáveis

In [ ]:
# Normalizando os dados
scaler = StandardScaler()
K_train_scaled = scaler.fit_transform(X_train)
K_test_scaled = scaler.transform(X_test)

# Treinamento dos modelos com dados normalizados
model_casa.fit(K_train_scaled, y_train_casa)
model_fora.fit(K_train_scaled, y_train_fora)

Aplicação dos tunnings de hiperparâmetros

In [ ]:
# Definindo o modelo
rf_model = RandomForestRegressor(random_state=42)

# Definindo a grade de hiperparâmetros
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Inicializando o GridSearchCV para a equipe da casa
grid_search_casa = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')
grid_search_fora = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# Treinamento dos modelos com GridSearchCV
grid_search_casa.fit(K_train_scaled, y_train_casa)
grid_search_fora.fit(K_train_scaled, y_train_fora)

# Melhores hiperparâmetros encontrados para o time da casa e o visitante
best_params_casa = grid_search_casa.best_params_
best_params_fora = grid_search_fora.best_params_

print(f"Melhores hiperparâmetros para o time da casa: {best_params_casa}")
print(f"Melhores hiperparâmetros para o time visitante: {best_params_fora}")

# Utilizando os melhores modelos
best_model_casa = grid_search_casa.best_estimator_
best_model_fora = grid_search_fora.best_estimator_

Métricas

In [ ]:
# Prever os valores para os dados de teste
y_pred_casa = best_model_casa.predict(K_test_scaled)
y_pred_fora = best_model_fora.predict(K_test_scaled)

# Avaliação do modelo para a equipe da casa
mse_casa = mean_squared_error(y_test_casa, y_pred_casa)
mae_casa = mean_absolute_error(y_test_casa, y_pred_casa)
r2_casa = r2_score(y_test_casa, y_pred_casa)
print(f'Modelo Casa (Tuned) - MSE: {mse_casa}, MAE: {mae_casa}, R2: {r2_casa}')

# Avaliação do modelo para a equipe visitante
mse_fora = mean_squared_error(y_test_fora, y_pred_fora)
mae_fora = mean_absolute_error(y_test_fora, y_pred_fora)
r2_fora = r2_score(y_test_fora, y_pred_fora)
print(f'Modelo Fora (Tuned) - MSE: {mse_fora}, MAE: {mae_fora}, R2: {r2_fora}')

Previsão do placar

In [ ]:
# IDs dos times selecionados para a partida
time1 = 11  # ID do time da casa
time2 = 14  # ID do time visitante

# Verificar se os times existem no conjunto de dados
if time1 not in dfTeams['common_name_encoded'].values or time2 not in dfTeams['common_name_encoded'].values:
    print('Um ou ambos os times não existem.')
else:
    # Selecionar as características do time da casa e do visitante
    time1_features = dfTeams[dfTeams['common_name_encoded'] == time1]
    time2_features = dfTeams[dfTeams['common_name_encoded'] == time2]

    # Garantir que tenhamos apenas uma linha por time
    time1_features = time1_features.iloc[0]
    time2_features = time2_features.iloc[0]

    # Criar o DataFrame de entrada para o time da casa
    novo_jogo_home = pd.DataFrame([{
        'losing_at_half_time_percentage': time1_features['losing_at_half_time_percentage'],
        'points_per_game_home': time1_features['points_per_game_home'],
        'minutes_per_goal_conceded': time1_features['minutes_per_goal_conceded'],
        'average_possession_away': time2_features['average_possession_home'],  # Usa a posse média de bola do time visitante
        'clean_sheets_home': time1_features['clean_sheets_home'],
        'goals_scored_per_match': time1_features['goals_scored_per_match'],
        'loss_percentage_ovearll': time1_features['loss_percentage_ovearll'],
        'minutes_per_goal_conceded_home': time1_features['minutes_per_goal_conceded_home'],
        'goal_difference': time1_features['goal_difference'],
        'goals_conceded_per_match_half_time': time2_features['goals_conceded_per_match_half_time'],  # Usa estatísticas defensivas do visitante
        'goals_conceded_home': time1_features['goals_conceded_home'],
        'fts_half_time_percentage': time1_features['fts_half_time_percentage'],
        'win_percentage': time1_features['win_percentage'],
        'total_goal_count': time1_features['total_goal_count'],
        'wins_home': time1_features['wins_home'],
        'losses': time1_features['losses'],
        'over35_percentage': time1_features['over35_percentage'],
        'minutes_per_goal_scored': time1_features['minutes_per_goal_scored'],
        'losing_at_half_time': time1_features['losing_at_half_time'],
        'wins': time1_features['wins']
    }])

    # Criar o DataFrame de entrada para o time visitante
    novo_jogo_away = pd.DataFrame([{
        'losing_at_half_time_percentage': time2_features['losing_at_half_time_percentage'],
        'points_per_game_home': time2_features['points_per_game_away'],  # Aqui estamos usando estatísticas do time visitante
        'minutes_per_goal_conceded': time2_features['minutes_per_goal_conceded'],
        'average_possession_away': time1_features['average_possession_away'],  # Usa a posse média de bola do time da casa
        'clean_sheets_home': time2_features['clean_sheets_away'],
        'goals_scored_per_match': time2_features['goals_scored_per_match'],
        'loss_percentage_ovearll': time2_features['loss_percentage_ovearll'],
        'minutes_per_goal_conceded_home': time2_features['minutes_per_goal_conceded_away'],
        'goal_difference': time2_features['goal_difference'],
        'goals_conceded_per_match_half_time': time1_features['goals_conceded_per_match_half_time'],  # Usa estatísticas defensivas do time da casa
        'goals_conceded_home': time2_features['goals_conceded_away'],
        'fts_half_time_percentage': time2_features['fts_half_time_percentage'],
        'win_percentage': time2_features['win_percentage'],
        'total_goal_count': time2_features['total_goal_count'],
        'wins_home': time2_features['wins_away'],
        'losses': time2_features['losses'],
        'over35_percentage': time2_features['over35_percentage'],
        'minutes_per_goal_scored': time2_features['minutes_per_goal_scored'],
        'losing_at_half_time': time2_features['losing_at_half_time'],
        'wins': time2_features['wins']
    }])

    # Fazer previsões usando os modelos treinados
    gols_casa_predito = model_casa.predict(novo_jogo_home)[0]
    gols_fora_predito = model_fora.predict(novo_jogo_away)[0]

    print(f'Previsão de Gols Casa: {gols_casa_predito:.2f}')
    print(f'Previsão de Gols Fora: {gols_fora_predito:.2f}')

    # Arredondar para obter o placar final
    placar_casa = round(gols_casa_predito)
    placar_fora = round(gols_fora_predito)

    print(f'Placar Previsto: {placar_casa} x {placar_fora}')


# KNN TEAMS

Define as variáveis que serão utilizadas para treinar o modelo

In [40]:
# Lista de características selecionadas
X = dfTeams[[ 
    'losing_at_half_time_percentage',   # Estatística do time
    'points_per_game_home',             # Estatística do time
    'minutes_per_goal_conceded',        # Estatística do time
    'average_possession_away',          # Estatística do time
    'clean_sheets_home',                # Estatística do time
    'goals_scored_per_match',           # Estatística do time
    'loss_percentage_ovearll',          # Estatística do time
    'minutes_per_goal_conceded_home',   # Estatística do time
    'goal_difference',                  # Estatística do time
    'goals_conceded_per_match_half_time', # Estatística do time
    'goals_conceded_home',              # Estatística do time
    'fts_half_time_percentage',         # Estatística do time
    'win_percentage',                   # Estatística do time
    'total_goal_count',                 # Estatística do time
    'wins_home',                        # Estatística do time
    'losses',                           # Estatística do time
    'over35_percentage',                # Estatística do time
    'minutes_per_goal_scored',          # Estatística do time
    'losing_at_half_time',              # Estatística do time
    'wins'                              # Estatística do time
]]


Define as colunas alvo

In [ ]:
# Alvos para gols marcados (home) e gols concedidos (away)
y_home = dfTeams['goals_scored']
y_away = dfTeams['goals_conceded']

Divide e normaliza os dados de treino

In [42]:
# Dividindo os dados de treino e teste
X_train_home, X_test_home, y_train_home, y_test_home = train_test_split(X, y_home, test_size=0.2, random_state=42)
X_train_away, X_test_away, y_train_away, y_test_away = train_test_split(X, y_away, test_size=0.2, random_state=42)

# Normalizando os dados
scaler = StandardScaler()
X_train_home = scaler.fit_transform(X_train_home)
X_test_home = scaler.transform(X_test_home)
X_train_away = scaler.fit_transform(X_train_away)
X_test_away = scaler.transform(X_test_away)

Tunnings de hiperparâmetros

In [43]:
# Definindo o modelo KNN
knn_model_home = KNeighborsRegressor()
knn_model_away = KNeighborsRegressor()

# Definindo os hiperparâmetros para o GridSearchCV
param_grid = {
    'n_neighbors': [3, 5, 7, 9],  # Número de vizinhos a considerar
    'weights': ['uniform', 'distance'],  # Tipo de ponderação das distâncias
    'metric': ['euclidean', 'manhattan', 'minkowski']  # Tipo de métrica de distância
}

# Grid Search para o time da casa
grid_search_home = GridSearchCV(knn_model_home, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search_home.fit(X_train_home, y_train_home)

# Melhor modelo para o time da casa
best_knn_home = grid_search_home.best_estimator_

# Grid Search para o time visitante
grid_search_away = GridSearchCV(knn_model_away, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search_away.fit(X_train_away, y_train_away)

# Melhor modelo para o time visitante
best_knn_away = grid_search_away.best_estimator_

Resultados dos tunnings de hiperparâmetros

In [ ]:
# Fazendo previsões usando os melhores modelos
y_pred_home_knn = best_knn_home.predict(X_test_home)
y_pred_away_knn = best_knn_away.predict(X_test_away)

# Avaliando o modelo
mse_home_knn = mean_squared_error(y_test_home, y_pred_home_knn)
r2_home_knn = r2_score(y_test_home, y_pred_home_knn)

mse_away_knn = mean_squared_error(y_test_away, y_pred_away_knn)
r2_away_knn = r2_score(y_test_away, y_pred_away_knn)

# Resultados após tuning
print(f"Melhor Modelo KNN (Casa) - MSE: {mse_home_knn}, R2: {r2_home_knn}")
print(f"Melhor Modelo KNN (Fora) - MSE: {mse_away_knn}, R2: {r2_away_knn}")

# Melhor hiperparâmetros
print(f"Melhores Hiperparâmetros (Casa): {grid_search_home.best_params_}")
print(f"Melhores Hiperparâmetros (Fora): {grid_search_away.best_params_}")

Previsão do placar

In [ ]:
# IDs dos times selecionados para a partida
time1 = 11  # ID do time da casa
time2 = 14  # ID do time visitante

# Verificar se os times existem no conjunto de dados
if time1 not in dfTeams['common_name_encoded'].values or time2 not in dfTeams['common_name_encoded'].values:
    print('Um ou ambos os times não existem.')
else:
    # Selecionar as características do time da casa e do visitante
    time1_features = dfTeams[dfTeams['common_name_encoded'] == time1]
    time2_features = dfTeams[dfTeams['common_name_encoded'] == time2]

    # Garantir que tenhamos apenas uma linha por time
    time1_features = time1_features.iloc[0]
    time2_features = time2_features.iloc[0]

    # Criar o DataFrame de entrada para o time da casa
    novo_jogo_home_knn = pd.DataFrame([{
        'losing_at_half_time_percentage': time1_features['losing_at_half_time_percentage'],
        'points_per_game_home': time1_features['points_per_game_home'],
        'minutes_per_goal_conceded': time1_features['minutes_per_goal_conceded'],
        'average_possession_away': time2_features['average_possession_home'],  # Usa a posse média de bola do time visitante
        'clean_sheets_home': time1_features['clean_sheets_home'],
        'goals_scored_per_match': time1_features['goals_scored_per_match'],
        'loss_percentage_ovearll': time1_features['loss_percentage_ovearll'],
        'minutes_per_goal_conceded_home': time1_features['minutes_per_goal_conceded_home'],
        'goal_difference': time1_features['goal_difference'],
        'goals_conceded_per_match_half_time': time2_features['goals_conceded_per_match_half_time'],  # Usa estatísticas defensivas do visitante
        'goals_conceded_home': time1_features['goals_conceded_home'],
        'fts_half_time_percentage': time1_features['fts_half_time_percentage'],
        'win_percentage': time1_features['win_percentage'],
        'total_goal_count': time1_features['total_goal_count'],
        'wins_home': time1_features['wins_home'],
        'losses': time1_features['losses'],
        'over35_percentage': time1_features['over35_percentage'],
        'minutes_per_goal_scored': time1_features['minutes_per_goal_scored'],
        'losing_at_half_time': time1_features['losing_at_half_time'],
        'wins': time1_features['wins']
    }])

    # Criar o DataFrame de entrada para o time visitante
    novo_jogo_away_knn = pd.DataFrame([{
        'losing_at_half_time_percentage': time2_features['losing_at_half_time_percentage'],
        'points_per_game_home': time2_features['points_per_game_away'],  # Aqui estamos usando estatísticas do time visitante
        'minutes_per_goal_conceded': time2_features['minutes_per_goal_conceded'],
        'average_possession_away': time1_features['average_possession_away'],  # Usa a posse média de bola do time da casa
        'clean_sheets_home': time2_features['clean_sheets_away'],
        'goals_scored_per_match': time2_features['goals_scored_per_match'],
        'loss_percentage_ovearll': time2_features['loss_percentage_ovearll'],
        'minutes_per_goal_conceded_home': time2_features['minutes_per_goal_conceded_away'],
        'goal_difference': time2_features['goal_difference'],
        'goals_conceded_per_match_half_time': time1_features['goals_conceded_per_match_half_time'],  # Usa estatísticas defensivas do time da casa
        'goals_conceded_home': time2_features['goals_conceded_away'],
        'fts_half_time_percentage': time2_features['fts_half_time_percentage'],
        'win_percentage': time2_features['win_percentage'],
        'total_goal_count': time2_features['total_goal_count'],
        'wins_home': time2_features['wins_away'],
        'losses': time2_features['losses'],
        'over35_percentage': time2_features['over35_percentage'],
        'minutes_per_goal_scored': time2_features['minutes_per_goal_scored'],
        'losing_at_half_time': time2_features['losing_at_half_time'],
        'wins': time2_features['wins']
    }])

    # Normalizar os dados de entrada do novo jogo (com o mesmo scaler usado no treino)
    novo_jogo_home_knn_scaled = scaler.transform(novo_jogo_home_knn)
    novo_jogo_away_knn_scaled = scaler.transform(novo_jogo_away_knn)

    # Fazer previsões usando os melhores modelos ajustados no GridSearch
    gols_casa_predito_knn = best_knn_home.predict(novo_jogo_home_knn_scaled)[0]
    gols_fora_predito_knn = best_knn_away.predict(novo_jogo_away_knn_scaled)[0]

    print(f'Previsão de Gols Casa (KNN): {gols_casa_predito_knn:.2f}')
    print(f'Previsão de Gols Fora (KNN): {gols_fora_predito_knn:.2f}')

    # Arredondar para obter o placar final
    placar_casa_knn = round(gols_casa_predito_knn)
    placar_fora_knn = round(gols_fora_predito_knn)

    print(f'Placar Previsto (KNN): {placar_casa_knn} x {placar_fora_knn}')


# Regressao Linear Nova tentativa

Com base nos modelos anteriores, refizemos o teste com a regressão linear utilizando uma nova abordagem

Divisão entre treinamento e teste

In [46]:
# Dividindo os dados em conjuntos de treinamento e teste
X_train_home, X_test_home, y_train_home, y_test_home = train_test_split(X, y_home, test_size=0.2, random_state=42)
X_train_away, X_test_away, y_train_away, y_test_away = train_test_split(X, y_away, test_size=0.2, random_state=42)

Normalização dos dados

In [47]:
# Escalando os dados
scaler = MinMaxScaler()
X_train_home_scaled = scaler.fit_transform(X_train_home)
X_test_home_scaled = scaler.transform(X_test_home)
X_train_away_scaled = scaler.fit_transform(X_train_away)
X_test_away_scaled = scaler.transform(X_test_away)

# Definindo o modelo de Regressão Linear
lr_home = LinearRegression()
lr_away = LinearRegression()


Tunning de hiperparâmetros

In [ ]:
# Definindo os hiperparâmetros para o GridSearchCV
param_grid = {
    'fit_intercept': [True, False],  # Se deve ajustar o intercepto
    'copy_X': [True, False]          # Se deve copiar X ou não (para desempenho)
}

# Grid Search para o modelo de gols em casa
grid_search_home = GridSearchCV(lr_home, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search_home.fit(X_train_home_scaled, y_train_home)

# Melhor modelo para gols em casa
best_lr_home = grid_search_home.best_estimator_

# Grid Search para o modelo de gols fora
grid_search_away = GridSearchCV(lr_away, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search_away.fit(X_train_away_scaled, y_train_away)

# Melhor modelo para gols fora
best_lr_away = grid_search_away.best_estimator_

# Treinando os melhores modelos
best_lr_home.fit(X_train_home_scaled, y_train_home)
best_lr_away.fit(X_train_away_scaled, y_train_away)

Métricas

In [ ]:
# Fazer previsões de gols para o time da casa e visitante
y_pred_home_lr = best_lr_home.predict(X_test_home_scaled)
y_pred_away_lr = best_lr_away.predict(X_test_away_scaled)

# Avaliando o modelo
mse_home_lr = mean_squared_error(y_test_home, y_pred_home_lr)
r2_home_lr = r2_score(y_test_home, y_pred_home_lr)

mse_away_lr = mean_squared_error(y_test_away, y_pred_away_lr)
r2_away_lr = r2_score(y_test_away, y_pred_away_lr)

# Exibir resultados de MSE e R2 após tuning
print(f"Modelo Regressão Linear (Casa) - MSE: {mse_home_lr:.2f}, R2: {r2_home_lr:.2f}")
print(f"Modelo Regressão Linear (Fora) - MSE: {mse_away_lr:.2f}, R2: {r2_away_lr:.2f}")

Previsão do placar

In [ ]:
# Previsão do placar para uma partida
time1 = 11  # ID do time da casa
time2 = 16  # ID do time visitante

# Verificar se os times existem no conjunto de dados
if time1 not in dfTeams['common_name_encoded'].values or time2 not in dfTeams['common_name_encoded'].values:
    print('Um ou ambos os times não existem.')
else:
    # Selecionar as características do time da casa e do visitante
    time1_features = dfTeams[dfTeams['common_name_encoded'] == time1]
    time2_features = dfTeams[dfTeams['common_name_encoded'] == time2]

    # Garantir que tenhamos apenas uma linha por time
    time1_features = time1_features.iloc[0]
    time2_features = time2_features.iloc[0]

    # Criar o DataFrame de entrada para o time da casa
    novo_jogo_home = pd.DataFrame([{
        'losing_at_half_time_percentage': time1_features['losing_at_half_time_percentage'],
        'points_per_game_home': time1_features['points_per_game_home'],
        'minutes_per_goal_conceded': time1_features['minutes_per_goal_conceded'],
        'average_possession_away': time2_features['average_possession_home'],
        'clean_sheets_home': time1_features['clean_sheets_home'],
        'goals_scored_per_match': time1_features['goals_scored_per_match'],
        'loss_percentage_ovearll': time1_features['loss_percentage_ovearll'],
        'minutes_per_goal_conceded_home': time1_features['minutes_per_goal_conceded_home'],
        'goal_difference': time1_features['goal_difference'],
        'goals_conceded_per_match_half_time': time2_features['goals_conceded_per_match_half_time'],
        'goals_conceded_home': time1_features['goals_conceded_home'],
        'fts_half_time_percentage': time1_features['fts_half_time_percentage'],
        'win_percentage': time1_features['win_percentage'],
        'total_goal_count': time1_features['total_goal_count'],
        'wins_home': time1_features['wins_home'],
        'losses': time1_features['losses'],
        'over35_percentage': time1_features['over35_percentage'],
        'minutes_per_goal_scored': time1_features['minutes_per_goal_scored'],
        'losing_at_half_time': time1_features['losing_at_half_time'],
        'wins': time1_features['wins']
    }])

    # Criar o DataFrame de entrada para o time visitante
    novo_jogo_away = pd.DataFrame([{
        'losing_at_half_time_percentage': time2_features['losing_at_half_time_percentage'],
        'points_per_game_home': time2_features['points_per_game_away'],
        'minutes_per_goal_conceded': time2_features['minutes_per_goal_conceded'],
        'average_possession_away': time1_features['average_possession_away'],
        'clean_sheets_home': time2_features['clean_sheets_away'],
        'goals_scored_per_match': time2_features['goals_scored_per_match'],
        'loss_percentage_ovearll': time2_features['loss_percentage_ovearll'],
        'minutes_per_goal_conceded_home': time2_features['minutes_per_goal_conceded_away'],
        'goal_difference': time2_features['goal_difference'],
        'goals_conceded_per_match_half_time': time1_features['goals_conceded_per_match_half_time'],
        'goals_conceded_home': time2_features['goals_conceded_away'],
        'fts_half_time_percentage': time2_features['fts_half_time_percentage'],
        'win_percentage': time2_features['win_percentage'],
        'total_goal_count': time2_features['total_goal_count'],
        'wins_home': time2_features['wins_away'],
        'losses': time2_features['losses'],
        'over35_percentage': time2_features['over35_percentage'],
        'minutes_per_goal_scored': time2_features['minutes_per_goal_scored'],
        'losing_at_half_time': time2_features['losing_at_half_time'],
        'wins': time2_features['wins']
    }])

    # Escalar os dados da nova partida
    novo_jogo_home_scaled = scaler.transform(novo_jogo_home)
    novo_jogo_away_scaled = scaler.transform(novo_jogo_away)

    # Fazer previsões usando os modelos treinados
    gols_casa_predito = best_lr_home.predict(novo_jogo_home_scaled)[0]
    gols_fora_predito = best_lr_away.predict(novo_jogo_away_scaled)[0]

    print(f'Previsão de Gols Casa: {gols_casa_predito:.2f}')
    print(f'Previsão de Gols Fora: {gols_fora_predito:.2f}')

    # Arredondar para obter o placar final
    placar_casa = round(gols_casa_predito)
    placar_fora = round(gols_fora_predito)

    print(f'Placar Previsto: {placar_casa} x {placar_fora}')
    print(time2_features['clean_sheets_away'])

Com base nesse notebook, conseguimos concluir que a tabela Matches conseguia trazer previsões mais realistas. Entretanto, as métricas dos modelos que utilizaram a tabela Teams parecem melhor, apesar dos placares absurdos.
O objetivo desse arquivo era gerar várias métricas e resultados que poderiam ser analisadas futuramente. Conseguimos perceber que há futuro no modelo de Regressão Linear, mas ainda é necessário verificar qual a melhor tabela a ser utilizada. A tabela Teams está com previsões de placares muito altas, porém, pode haver algum erro na normalização dos dados que fez com que os resultados fossem tão elásticos. Com isso, a comparação foi concluída e está melhor documentada na documentacao.md, mas o modelo ainda passará por verificações futuras

Com base nas comparações dos três últimos algoritmos, todos os modelos possuem boas métricas para o time da casa, mas não tão boas para o de fora. Segue os resultados obtidos:
Random Forest: Possui métricas para gols fora de casa ruins, como um R² negativo
KNN: Possui uma métrica de gols fora ok, como um R² de 0,3
Regressão Linear: Possui ótimas métricas de gols fora

*Geração da explicabilidade do modelo de Regressão Linear*

Grafico Home

In [ ]:
import shap

# Criando o explicador SHAP para o time da casa
explainer_home = shap.LinearExplainer(best_lr_home, X_train_home_scaled)
shap_values_home = explainer_home.shap_values(X_test_home_scaled)

# Plot SHAP summary para o time da casa
shap.summary_plot(shap_values_home, X_test_home_scaled, feature_names=features_home)



Grafico Away

In [ ]:
# Criando o explicador SHAP para o time visitante
explainer_away = shap.LinearExplainer(best_lr_away, X_train_away_scaled)
shap_values_away = explainer_away.shap_values(X_test_away_scaled)

# Plot SHAP summary para o time visitante
shap.summary_plot(shap_values_away, X_test_away_scaled, feature_names=features_away)

O gráfico SHAP visualiza como cada característica de um modelo de aprendizado de máquina contribui para suas previsões, permitindo uma compreensão clara do impacto individual de cada feature nas previsões feitas pelo modelo. No eixo Y estão listadas as características, enquanto o eixo X representa os valores SHAP, que medem a contribuição de cada feature para as previsões. Cada bolinha no gráfico representa uma observação específica do conjunto de dados, com sua posição ao longo do eixo X indicando se a característica teve um efeito positivo ou negativo na previsão. A cor das bolinhas frequentemente reflete o valor da característica, ajudando a identificar padrões de impacto e a variabilidade entre as observações. 
Através desses gráficos gerados, é possível perceber quais váriaveis estão impactando positivamente ou negativamente o modelo.


Explicação dos resultados da explicabilidade da previsões de goals_home

| Características                                     | Descrição                            |
|-----------------------------------------------------|-------------------------------------|
| total_goal_count                                    |Todas as bolinhas estão no lado positivo, ou seja, todas impactam positivamente|
| goal_difference                                     |Três bolinhas estão do lado positivo, indicando um bom impacto no modelo|
| goals_scored_per_match                              |Parece equilibrado, já que duas bolinhas se aproximam bastante do zero e as outras duas estão uma de cada lado|
| wins                                                |Apresenta três bolinhas do lado negativo, indicando impacto negativo|
| minutes_per_goal_conceded                           |Apresenta todas as bolinhas do lado negativo|


Explicação dos resultados da explicabilidade da previsões de goals_away

| Características                                     | Descrição                            |
|-----------------------------------------------------|-------------------------------------|
| total_goal_count                                    |Todas as bolinhas estão no lado positivo, ou seja, todas impactam positivamente|
| goal_difference                                     |Três bolinhas estão do lado positivo, indicando um bom impacto no modelo|
| goals_scored_per_match                              |Parece equilibrado, já que duas bolinhas se aproximam bastante do zero e as outras duas estão uma de cada lado|
| minutes_per_goal_conceded                      |Apresenta todas as bolinhas do lado positivo, o que indica impacto positivo|
| losses | Parece equilibrado, já que duas bolinhas se aproximam bastante do zero e as outras duas estão uma de cada lado

Acima foram demonstradas apenas as melhores 5 features para cada previsão, mas através do gráfico é possível perceber o quanto todas as features impactam no modelo.